## FFM Application with Python

### 1. Create Test Signals
To study how FFT works, I first created code to generate multiple sine signals using the following general form of a sine wave.
$$y = A \sin(2\pi f t + \phi)$$

In [86]:
import numpy as np
import os
import librosa
import IPython.display as ipd 
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [87]:
class sinWave:
    def __init__(self, **kwargs):
        self.A = kwargs.get('amp', 1)
        self.f = kwargs.get('freq', 1)
        self.p = kwargs.get('phase_shift', 0)
        self.endTime = kwargs.get('endTime', 1)
        self.sampleTime = kwargs.get('sampleTime', 0.01)

    def getDomain(self):
        return np.arange(0.0, self.endTime, self.sampleTime)
        
    def createSinWave(self,time):        
        return self.A * np.sin(2*np.pi*self.f*time + self.p)

The following code generate three different sine waves, each for the

$2sin(2\pi)$,

$5sin(3\cdot2\pi)$, and

$9sin(2\cdot2\pi)$.

In [88]:
sr = 1/22050 #standard 44.1 kHz
amplitude = 3
a4_freq = 440 
csharp5_freq = 554.365
e5_freq = 659.365

In [107]:
colors = dict(
    original = '#27aeef',
    noisy = '#ea5545',
    denoised = '#87bc45',
)

In [90]:
# generate sin waves
sin = sinWave(amp = amplitude, sampleTime=sr)
t = sin.getDomain()
sin.f = a4_freq
a4_y = sin.createSinWave(time=t)
sin.f = csharp5_freq
csharp5_y = sin.createSinWave(time=t)
sin.f = e5_freq
e5_y = sin.createSinWave(time=t)
a_chord = a4_y + csharp5_y + e5_y

#plot graph
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=t[:len(t)//20],
    y=a4_y[:len(t)//20],
    mode='lines',
    name='A4',
    line=dict(color='rgba(173, 216, 230, 0.6)'), 
))

fig.add_trace(go.Scatter(
    x=t[:len(t)//20],
    y=csharp5_y[:len(t)//20],
    mode='lines',
    name='C#5',
    line=dict(color='rgba(144, 238, 144, 0.6)'),  
))

fig.add_trace(go.Scatter(
    x=t[:len(t)//20],
    y=e5_y[:len(t)//20],
    mode='lines',
    name='E5',
    line=dict(color='rgba(255, 182, 193, 0.6) '), 
))

fig.add_trace(go.Scatter(
    x=t[:len(t)//20],
    y=a_chord[:len(t)//20],
    mode='lines',
    name='A Chord',
    line=dict(color=colors['original']) 
))

fig.update_layout(
    title=dict(
        text='A chord',
        x = 0.5,
        font_size=30
    ),
    font = dict(
        family='Raleway'
    ),
    xaxis_title=dict(
        text='Time(s)',
        font_size=20
    ),
    yaxis_title=dict(
        text='Amplitude',
        font_size=20
    ),
    legend=dict(
        font_size=15,
    )
)

fig.show()

In [91]:
a4_audio = ipd.Audio(data=a4_y, rate=1/sr)
csharp5_audio = ipd.Audio(data=csharp5_y, rate=1/sr)
e5_audio = ipd.Audio(data=e5_y, rate=1/sr)
ipd.Audio(data = a_chord, rate=1/sr)

In [102]:
noise_level = 0.7
noisy_a_chord = a_chord + noise_level * np.random.randn(len(a_chord))
sr = 1/22050 #standard 44.1 kHz
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=t[:len(t)//20],
    y=noisy_a_chord[:len(t)//20],
    mode='lines',
    name='noisy A chord',
    line=dict(color=colors['noisy']), 
))
fig.add_trace(go.Scatter(
    
    x=t[:len(t)//20],
    y=a_chord[:len(t)//20],
    mode='lines',
    name='original A chord',
    line=dict(color=colors['original']), 
))

fig.update_layout(
    title=dict(
        text='Noisy A chord',
        x = 0.5,
        font_size=30
    ),
    font = dict(
        family='Raleway'
    ),
    xaxis_title=dict(
        text='Time(s)',
        font_size=20
    ),
    yaxis_title=dict(
        text='Amplitude',
        font_size=20
    ),
    legend=dict(
        font_size=15,
    )
)
fig.show()
ipd.Audio(data = noisy_a_chord, rate=1/sr)

In [103]:
N = len(a_chord)               # number of sampling points
sr = 1/sin.sampleTime    # sampling rate (# samples per second)
T = N/sr                            # total time
k = np.arange(N)                    # index for frequency [0-(n-1)]
freq = k/T                          # plotting points

In [104]:
a_chord_fft = np.fft.fft(a_chord) / N * 2
a_chord_fft_ps = (np.abs(a_chord_fft)**2) * (1/N**2)
a_chord_fft_ps_one_sided = a_chord_fft_ps[:N//2].copy()

noisy_a_chord_fft = np.fft.fft(noisy_a_chord) / N * 2
noisy_a_chord_fft_ps = (np.abs(noisy_a_chord_fft)**2) * (1/N**2)
noisy_a_chord_fft_ps_one_sided = noisy_a_chord_fft_ps[:N//2].copy()


In [105]:
graph_a_chord_fft = go.Figure()
graph_a_chord_fft.add_trace(go.Scatter(
    x=freq[:1000],
    y=np.real(np.abs(a_chord_fft[:1000])),
    mode='markers+lines',
    name='A chord FFT',
    line=dict(color=colors['original']), 
))

graph_noisy_a_chord_fft = go.Figure()
graph_noisy_a_chord_fft.add_trace(go.Scatter(
    x=freq[:1000],
    y=np.real(np.abs(noisy_a_chord_fft[:1000])),
    mode='markers+lines',
    name='noisy A chord FFT',
    line=dict(color=colors['noisy']), 
))

graph_noisy_a_chord_fft_zooo_in = go.Figure()
graph_noisy_a_chord_fft_zooo_in.add_trace(go.Scatter(
    x=freq[600:750],
    y=np.real(np.abs(noisy_a_chord_fft[600:750])),
    mode='markers+lines',
    name='noisy A chord FFT zoomed in',
    line=dict(color=colors['noisy']), 
))

combined_graphs = make_subplots(rows=1, cols=3, subplot_titles=('A Chord FFT', 'Noisy A Chord FFT', 'noisy A chord FFT zoomed in'))
combined_graphs.add_traces(graph_a_chord_fft.data, rows=1, cols=1)
combined_graphs.add_traces(graph_noisy_a_chord_fft.data, rows=1, cols=2)
combined_graphs.add_traces(graph_noisy_a_chord_fft_zooo_in.data, rows=1, cols=3)
combined_graphs.update_layout(
    legend=dict(
    orientation='h',
    xanchor='center',
    x = 0.5,
    y = -0.1,
    font_size=15
),
title=dict(text='A Chord FFT in Magnitude Spectrum',
x = 0.5,
xanchor='center',
font_size = 30
),
font=dict(
    family='Railway'
)
)

In [169]:
full_mask = abs(noisy_a_chord_fft) > 0.08
fhat = full_mask * noisy_a_chord_fft

graph_denoised_a_chord_fft = go.Figure()
graph_denoised_a_chord_fft.add_trace(go.Scatter(
    x=freq[:1000],
    y=np.real(np.abs(fhat[:1000])),
    mode='markers+lines',
    name='A chord FFT',
    line=dict(color=colors['denoised']), 
))
graph_denoised_a_chord_fft.update_layout(
    title=dict(
        text='Denoised FFT',
        xanchor='center',
        x = 0.5,
        font_size=30,
    ),
    xaxis_title=dict(text='Frequency(Hz)', font_size=20),
    yaxis_title=dict(text='Amplitude', font_size=20),
    font=dict(
    family='Railway'
)
)
graph_denoised_a_chord_fft.show()
denoised_achord_ifft = np.fft.ifft(fhat * N / 2)


In [170]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=t[:len(t)//20],
    y=noisy_a_chord[:len(t)//20],
    mode='lines',
    name='noisy A chord',
    line=dict(color=colors['noisy']), 
))
fig.add_trace(go.Scatter(
    x=t[:len(t)//20],
    y=a_chord[:len(t)//20],
    mode='lines',
    name='original A chord',
    line=dict(color=colors['original']), 
))
fig.add_trace(go.Scatter(
    x=t[:len(t)//20],
    y=np.real(denoised_achord_ifft[:len(t)//20]),
    mode='lines',
    name='Denoised A chord',
    line=dict(color=colors['denoised']), 
))

fig.update_layout(
    legend=dict(
    orientation='h',
    xanchor='center',
    x = 0.5,
    y = -0.1,
    font_size=15        
    ),
    title=dict(
        text='Filtered A chord sound (IFFT applied)',
        xanchor='center',
        x=0.5,
        font_size=30
    ),
    font_family='Railway'
)

In [ ]:
from IPython.display import display, HTML

audio_orig = ipd.Audio(a_chord, rate=sr)
audio_noisy = ipd.Audio(noisy_a_chord, rate=sr)
audio_filtered = ipd.Audio(data=np.real(denoised_achord_ifft), rate = sr)
html = f"""
<table>
  <tr>
    <th>Original</th>
    <th>Noisy</th>
    <th>Filtered</th>
  </tr>
  <tr>
    <td>{audio_orig._repr_html_()}</td>
    <td>{audio_noisy._repr_html_()}</td>
    <td>{audio_filtered._repr_html_()}</td>
  </tr>
</table>
"""
display(HTML(html))

Original,Noisy,Filtered
Your browser does not support the audio element.,Your browser does not support the audio element.,Your browser does not support the audio element.
